## Matching items with master-items  
#### Creating additional columns according to the degree of difference. The degree of difference is calculated by the method Damerau-Levenshtein distance represents the number of insertions, deletions, substitutions and transpositions required to change one string to another. A string that has several similar strings by the Damerau-Levenshtein method will sort by the Jaro-Winkler Similarity method

##### Notes:



In [1]:
from numpy import nan
import pandas as pd
# import re

import pyodbc
from sqlalchemy import create_engine
import urllib

from jellyfish import damerau_levenshtein_distance, jaro_winkler_similarity
from thefuzz.fuzz import token_sort_ratio

from operator import itemgetter
import functools
import operator
from math import ceil

#imports stopwords
# from nltk.corpus import stopwords
#imports stopwords

# import cleanco
from termdata import terms_by_type, terms_by_country #this termdata.py copy from lib cleanco

from tqdm import tqdm
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))
#
# pd.set_option('display.max_columns', None)

In [2]:
# # package version
# python    == 3.9.7
# numpy     == 1.20.3
# pandas    == 1.3.4
# re        == 2.2.1
# jellyfish == 0.9.0
# cleanco   == 2.2

In [2]:
def creating_stop_words():
    """Creating list of stop words"""
    # # Creat list of stop words _ first variant
    # # Import words from lib clenco
    ts = functools.reduce(operator.iconcat, terms_by_type.values(), [])
    cs = functools.reduce(operator.iconcat, terms_by_country.values(), [])
    terms_stop_words = set(ts + cs)
    
    # # Creat list of stop words _ first variant + preprocessing by lib clenco
    # # this variant was used at the beginning, but there was a bug that removes the meaning of the word
    # terms = cleanco.prepare_default_terms()
    # stop_words = ['and']
    # terms_stop_words = {ii for i in map(itemgetter(1), terms) for ii in i}
    # terms_stop_words |= set(stop_words) #update set e.g. set.update()

    # stopwords_set = set(stopwords.words('english'))
    
    return terms_stop_words

def remove_stop_words(string):
    """Remove stop words from string """
    list_stop_words = creating_stop_words()
    return ' '.join(filter(lambda x: x not in list_stop_words, string.split()))

def normalize(df):
    """The column 'clean' is created with normalized names, for example,
        everything except letters and numbers are deleted ; business stop words are deleted"""

    df = df.copy()
    df['clean'] = df.iloc[:,0].str.lower()
    lower_name = df['clean'].copy()
    
    df['clean'].replace(to_replace ='[\W]+', value = ' ', regex = True, inplace=True)
    df['clean'] = df['clean'].apply(remove_stop_words)
    df['clean'].replace(to_replace ='[\W]+', value = '', regex = True, inplace=True)
    df['clean'] = df['clean'].apply(remove_stop_words) #because after deleting '[\W]+' maybe will find pattern remove_stop_words, need two times remove_stop_words
    df['clean'].replace(to_replace ='[\W]+', value = '', regex = True, inplace=True)


    index_row_with_stopwords = df[df['clean'] != lower_name].index
    print(f"count row with stop words: {len(index_row_with_stopwords)}")
    
    # df['clean'].replace(to_replace =' ', value = '', regex = True, inplace=True)
    # df['clean'] = df['clean'].str.strip()

    return df#, index_row_with_stopwords


def match(df_vendor, df_master, porog=3, metric_similarity='t_s_r', debug=False):
    """Matching name from dataframe_1 to name from dataframe_2
       'porog' is max edit-distance, 'metric_similarity' is method calculate edit-distance
       (t_s_r - token_sort_ratio, d_l - damerau_levenshtein """
    df_vendor = normalize(vendor_manuf)
    df_master = normalize(DKH_manuf)

    result = pd.DataFrame()
    data_ =[]
    columns_ = ['id']
    columns_.extend([column for column in df_vendor.columns])
    differ_degree = [f"differ_{n}" for n in range(porog+1)]
    columns_.extend(differ_degree)

    for row_ven in df_vendor.itertuples():
        # create the dictionary with keys or clearing (dict.fromkeys(differ_degree, []) don't work)
        scores = dict()
        for key in differ_degree:
            scores[key] = []
         
        for id, name, clean in df_master.itertuples():
 ############### START BLOCK calculate method ######################
            if metric_similarity == 'd_l':
                score_primary = damerau_levenshtein_distance(row_ven[2], clean)
                if score_primary <= porog:
                    score_secondary = round(jaro_winkler_similarity(row_ven[2], clean), 3)
                    differ_n = f"differ_{score_primary}"
                    scores[differ_n].append(tuple([clean, score_secondary, name, id]))
            
            elif metric_similarity == 't_s_r':
                score_primary = token_sort_ratio(row_ven[2], clean)
                score_primary_scale = ceil((100 - score_primary) / 10) # rounding up
                if score_primary_scale <= porog:
                    score_secondary = score_primary
                    differ_n = f"differ_{score_primary_scale}"
                    scores[differ_n].append(tuple([clean, score_secondary, name, id]))
                
            else:
                return print('Error choise metric_similarity')
 ############### END BLOCK calculate method #######################           
            
        row_data=list(row_ven)
        #sorted dict by score_secondary
        for key in sorted(scores):
            differ_n = sorted(scores[key], key=itemgetter(1), reverse=True)
            row_data.append(differ_n)
        data_.append(tuple(row_data))

        result = pd.DataFrame(data=data_, columns=columns_)

        #NEED to make protection
        # if result.empty:

        #define empty cell(empty list) as np.nan
        for i in range(3,result.shape[1]):
            result.iloc[:,i] = result.iloc[:,i].apply(lambda x: nan if len(x)==0 else x)
    
    if debug:
        return result
    else:
        return split_df(result, start=3, end=5, max_columns=3)

def split_df(df, start, end, max_columns, extract_name_id=True):
    """
    extract_name_id e.g. from (elkay, 0.907, Elkay, 697) extract to two columns Elkay 697
    """
    df_result = df.iloc[:,:2].copy()
    for N in range(start, end+1):
        df_temp = df.iloc[:,N].apply(pd.Series)
        df_temp = df_temp.iloc[:,:max_columns]
        count_columns = df_temp.shape[1]
        name = df.iloc[:,N].name
        df_temp.columns = [name + '_' + str(n) for n in range(count_columns)]
        if extract_name_id:
            for column in df_temp.items():
                df_sub_temp = column[1].apply(pd.Series).iloc[:,2:4] #hard range(2,4) 2-original name, 3-id
                if df_sub_temp.empty:
                    continue
                df_sub_temp.columns = [column[0] + '_name', column[0] + '_id']
                df_result = pd.concat([df_result, df_sub_temp], axis=1)
        else:
            df_result = pd.concat([df_result, df_temp], axis=1)
    
    return df_result.convert_dtypes()
print('function declaration')

function declaration


In [4]:
tt = creating_stop_words()
'brands' in tt

True

In [3]:
%%time
# select products vendor
vendor_id = 127
print(pd.Timestamp.now())
params_VM = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                    "SERVER=name;"
                                    "DATABASE=name;"
                                    "UID=name;"
                                    "PWD=name")
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_VM}")

query = f"""
            SELECT m.id, m.name 
            FROM manufacturers m
            LEFT JOIN manufacturers_to_master_map mtmm ON mtmm.manufacturer_id = m.id 
            WHERE vendor_id = {vendor_id} AND mtmm.master_manufacturer_id IS NULL;
        """

vendor_manuf = pd.read_sql(query, con=engine)
engine.dispose()
vendor_manuf.to_csv('vendor_manufacturers.zip', index=False)
vendor_manuf.info(memory_usage='deep')

vendor_manuf.index = vendor_manuf.loc[:, 'id']
vendor_manuf.drop(columns=['id'], inplace=True)
print(pd.Timestamp.now())

2023-10-25 09:25:37.942655
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2 non-null      int64 
 1   name    2 non-null      object
dtypes: int64(1), object(1)
memory usage: 292.0 bytes
2023-10-25 09:25:41.157055
CPU times: total: 0 ns
Wall time: 3.21 s


In [4]:
print(pd.Timestamp.now())
params_DK = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                    "SERVER=name;"
                                    "DATABASE=name;"
                                    "UID=name;"
                                    "PWD=name")

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params_DK}")

query = """
            SELECT m.manufacturer_ID as id, m.name
            FROM manufacturers m
        """

DKH_manuf = pd.read_sql(query, con=engine)
engine.dispose()
DKH_manuf.to_csv('DKH_manuf.zip', index=False)
DKH_manuf.info(memory_usage='deep')

DKH_manuf.index = DKH_manuf.loc[:, 'id']
DKH_manuf.drop(columns=['id'], inplace=True)
print(pd.Timestamp.now())

2023-10-25 09:25:41.449791
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10667 entries, 0 to 10666
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10667 non-null  int64 
 1   name    10667 non-null  object
dtypes: int64(1), object(1)
memory usage: 850.7 KB
2023-10-25 09:25:45.268202


In [5]:
vendor_manuf = normalize(vendor_manuf)
DKH_manuf = normalize(DKH_manuf)

count row with stop words: 2
count row with stop words: 8875


# MANUFACTURERS

In [5]:
# name = 'MERCHANDISING'
# display(manuf_mas[manuf_mas['name'].str.contains(name, case=False)])
# display(manuf_ven[manuf_ven['name'].str.contains(name, case=False)])

In [7]:
print(pd.Timestamp.now())
manuf_matching_dl = match(vendor_manuf, DKH_manuf, porog=3, metric_similarity='d_l', debug=True)
manuf_matching_dl.info()
print(pd.Timestamp.now())

2023-10-25 09:25:53.975590
count row with stop words: 2
count row with stop words: 8875
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        2 non-null      int64  
 1   name      2 non-null      object 
 2   clean     2 non-null      object 
 3   differ_0  1 non-null      object 
 4   differ_1  0 non-null      float64
 5   differ_2  1 non-null      object 
 6   differ_3  2 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 240.0+ bytes
2023-10-25 09:25:54.585151


In [8]:
manuf_matching_dl

,id,name,clean,differ_0,differ_1,differ_2,differ_3
0,50199,CURT MANUFACTURING (LIP,curtlip,NaN,NaN,"[(curtis, 0.91, CURTIS INDUSTRIES LLC, 28964)]","[(curt, 0.914, CURT MANUFACTURING, LLC, 34145)..."
1,50200,E-CLOTH INC,ecloth,"[(ecloth, 1.0, E-CLOTH INC, 31456)]",NaN,NaN,"[(eco, 0.867, Eco-Products, Inc, 26062), (eco,..."


In [8]:
print(pd.Timestamp.now())
manuf_matching_t_s_r = match(vendor_manuf, DKH_manuf, porog=3, metric_similarity='t_s_r', debug=False)
manuf_matching_t_s_r.info()
print(pd.Timestamp.now())

2023-08-28 13:09:49.637693
count row with stop words: 3
count row with stop words: 8822
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               6 non-null      Int64 
 1   name             6 non-null      string
 2   differ_0_0_name  1 non-null      string
 3   differ_0_0_id    1 non-null      Int64 
 4   differ_2_0_name  2 non-null      string
 5   differ_2_0_id    2 non-null      Int64 
 6   differ_2_1_name  1 non-null      string
 7   differ_2_1_id    1 non-null      Int64 
 8   differ_2_2_name  1 non-null      string
 9   differ_2_2_id    1 non-null      Int64 
dtypes: Int64(5), string(5)
memory usage: 638.0 bytes
2023-08-28 13:09:50.929375


In [11]:
# manuf_matching_t_s_r#[manuf_matching_t_s_r['differ_2_0_id'].notna()]
manuf_matching_dl#[manuf_matching_t_s_r['differ_2_0_id'].notna()]

,id,name,differ_0_0_name,differ_0_0_id,differ_1_0_name,differ_1_0_id,differ_1_1_name,differ_1_1_id,differ_1_2_name,differ_1_2_id,differ_2_0_name,differ_2_0_id,differ_2_1_name,differ_2_1_id,differ_2_2_name,differ_2_2_id
0,50022,CORELLE,Corelle Brands LLC,29968,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Lorell,374,<NA>,<NA>,<NA>,<NA>
1,50023,CORINGWARE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,50025,DARRELL LEA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,50026,PRODUCTWORKS LLC,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,50027,PYREX,<NA>,<NA>,PYLEX,33064,PUREX,21299,PAREX USA INC,32327,Pyramex,29880,Pamex,2043,Norex,28
5,50028,VAN HOLTENS,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [10]:
manuf_matching_t_s_r[manuf_matching_t_s_r['name']=='SEYMOUR'].T

,1
id,48683
name,SEYMOUR
clean,seymour
differ_0,"[(seymour, 100, SEYMOUR MFG., 1658)]"
differ_1,NaN
differ_2,NaN
differ_3,"[(passseymour, 78, PASS & SEYMOUR, 29728), (se..."


In [11]:
col0 = ['differ_0_0_name','differ_0_1_name','differ_0_2_name']
col1 = ['differ_1_0_name','differ_1_1_name','differ_1_2_name']
manuf_matching_t_s_r[(manuf_matching_t_s_r[col1].notna().any(axis=1)) & (manuf_matching_t_s_r[col1].notna().any(axis=1))][['name']+col1].sort_values('name')#.iloc[80:120]

,name,differ_1_0_name,differ_1_1_name,differ_1_2_name
62,AFCO INDUSTRIES,FC INDUSTRIES,GAMCO INDUSTRIES,SANCO INDUSTRIES INC
22,ALLSTAR MARKETING GRP,ALLSTAR MARKETING GROUP,<NA>,<NA>
184,AMERICAN,AS AMERICA INC,<NA>,<NA>
209,ARIES MANUFACTURING,WARE MANUFACTURING INC,<NA>,<NA>
421,BK LLC SOUTHLAND,Southland,<NA>,<NA>
286,CP INDUSTRIES,LC Industries,C.L. INDUSTRIES INC.,MP INDUSTRIES
140,FOSS MANUFACTURING,BOSS MANUFACTURING COMPANY,<NA>,<NA>
768,GERNAT ENTERPRISES,"G.E.T. Enterprises, Inc",<NA>,<NA>
155,INCOM MANUFACTURING,ENCO MANUFACTURING CORP,<NA>,<NA>
561,IPG BUILDING PRODUCTS,CPG BUILDING PRODUCTS LLC,<NA>,<NA>


In [10]:
manuf_matching_t_s_r

,id,name,differ_1_0_name,differ_1_0_id,differ_1_1_name,differ_1_1_id,differ_1_2_name,differ_1_2_id,differ_2_0_name,differ_2_0_id,differ_2_1_name,differ_2_1_id,differ_2_2_name,differ_2_2_id
0,46611,STAY AWAY,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,46613,SALTON CANADA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,46614,IMAGE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,THE IMAGINE GROUP LLC,32078,<NA>,<NA>,<NA>,<NA>
3,46615,SHUR-TRIM,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,46624,HUSQVARNA CANADA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,48335,DELTA BROKERAGE CO,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
879,48336,TECH STEEL,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
880,48338,GRACIOUS LIVING CORPORA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,GRACIOUS LIVING CORP,31007,<NA>,<NA>,<NA>,<NA>
881,48339,COLUMBIA-MBF,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,COLUMBIA,2370,Columbian,26676,<NA>,<NA>


In [7]:
method1_manuf = manuf_matching_dl.loc[manuf_matching_dl['differ_0_0_id'].notna(), ['id', 'name', 'differ_0_0_name', 'differ_0_0_id']]
method2_manuf = manuf_matching_t_s_r.loc[manuf_matching_t_s_r['differ_0_0_id'].notna(), ['id', 'name', 'differ_0_0_name', 'differ_0_0_id']]

result_matching_manufacturers = pd.concat([method1_manuf, method2_manuf], keys=['m1', 'm2'])
result_matching_manufacturers.reset_index(inplace=True)
result_matching_manufacturers.drop(columns=['level_1'], inplace=True)
result_matching_manufacturers.drop_duplicates(subset=['id', 'differ_0_0_id'], keep='last', inplace=True)
result_matching_manufacturers = result_matching_manufacturers.astype({'id':'Int32', 'differ_0_0_id':'Int32'}) 


if (result_matching_manufacturers['id'].value_counts() > 1).any():
    print('Find more than one matching - ERROR')
else:
    print('Find one matching(one manuf_v-one manuf_m) - OK')


Find one matching(one manuf_v-one manuf_m) - OK


In [8]:
# manuf_matching_t_s_r.loc[manuf_matching_t_s_r['differ_0_0_id'].notna(), ['id', 'name', 'differ_0_0_name', 'differ_0_0_id']]
result_matching_manufacturers

,level_0,id,name,differ_0_0_name,differ_0_0_id
3,m2,49986,DUKE CANNON SUPPLY CO,DUKE CANNON SUPPLY COMPANY LLC,31453
4,m2,49990,INSTANT POWER,INSTANT POWER CORP,32639
5,m2,49991,MIDSTATE PLASTICS CORP,Midstate Plastics Corporation,32828


In [21]:
%%time

cnxn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}")
cursor = cnxn.cursor()
## insert data
query1 = f"INSERT INTO {table} VALUES (?, ?)"

for row in tqdm(result_matching_manufacturers.itertuples(index=False)):
    cursor.execute(query1, int(row.id), int(row.differ_0_0_id))

cnxn.commit()
cursor.close()
cnxn.close()
print(pd.Timestamp.now())

5it [00:00,  7.25it/s]


2023-04-20 13:22:34.694475
CPU times: total: 46.9 ms
Wall time: 1.98 s


In [29]:
# key:target id, values: list of secondary id simular manufacturers
"""e.q. you find information like 
Newark Electronics, sometimes called Newark element14, Newark Corporation, Newark an Avnet Company or Newark, is a Chicago-based electronic components distribution company
https://en.wikipedia.org/wiki/Newark_element14
i suggest fill information
"""
data_equal_manufacturers = {'Newark':['Newark Electronics', 'Newark element14', 'Newark Corporation', 'Newark an Avnet Company']}
# key: name, values: list of name secondary simular manufacturers
"""e.g. you find similar item and after thorough check its become not equal. And """
data_similar_manufacturers_not_equal = {'Altronix':['Alextronix']}

In [21]:
result_matching_manufacturers.drop(columns=['level_0']).to_csv(f'vendor_{vendor_id}_manuf.csv', index=False)

## Не выявленный матчинг 

In [3]:
## Набор тестов для нвого алгоритма

"""BLACK & DECKER/DEWALT
BOSMERE/PLANT STAND  => BOSMERE INC
CABOT/VALSPAR CORP               VALSPAR CORPORATION
CHERVON NA/SKIL                        Chervon North America inc.
CLOROX COMPANY, THE                     CLOROX CO
CUSTOM BLDG PRODUCTS                           Custom Building Products
DIAL MFG INC                                DIAL MANUFACTURING INC
DKB HOUSEHOLD USA/ZYLISS                       ZYLISS USA CORP
EATON J T                           JT EATON & CO
ELIDE FIRE USA CORPS.                  ELIDE FIRE USA CORP
FEDERAL MOGUL/CHAMP/WAGNER                  FEDERAL MOGUL CORP
FLINT & WALLING/STAR WATER    F&W - FLINT & WALLING
GLEASON INDUSTRIAL PRODUCTS : GLEASON INDUSTRIAL PRD
GRO-WELL BRANDS INC : GRO WELL BRANDS CP INC
GULF STREAM HOME & GARDEN : GULFSTREAM HOME & GARDEN INC
HARTZ MOUNTAIN CORP : HARTZ MOUNTAIN CORPORATON
HIGHLINE WARREN LLC : HIGHLINE WARREN/PLEWS
HONEY-CAN-DO INTERNATIONAL : HONEY CAN DO INTL INC
HD HUDSON MFG. : HUDSON, H D MFG CO
Imperial Manufacturing Group : IMPERIAL MFG GROUP USA INC
Vise-Grip / Irwin Industrial Tools : IRWIN INDUSTRIAL TOOL CO
LEBANON SEABOARD CORP. : LEBANON SEABOARD SEED CORP
JS PRODUCTS INC : J S PRODUCTS
Kohler : KOHLER/STERLING
Liquid Nails : LIQUID NAILS/PPG ARCH FIN
Little Giant : LITTLE GIANT/FRANKLIN ELECTRIC
Martin Wheel : MARTIN WHEEL CO., INC., THE
Meguiar's : MEGUIARS INC
THE METAL WARE CORPORATION : METAL WARE CORP, THE
METHOD PROD : METHOD PRODUCTS PBC
MIDWEST AIR TECHNOLOGIES INC : MIDWEST AIR TECH/IMPORT
MID-WEST METAL PRODUCTS : MIDWEST METAL PRODUCTS CO INC
Motsenbocker's Liftoff : MOTSENBOCKER LIFT-OFF
NEWBORN BROTHERS & CO INC : NEWBORN BROS & CO INC
Norton Abrasives : NORTON ABRASIVES/ST GOBAIN
NYL HOLDINGS LLC : NYL HOLDINGS LLC/WESTCLOX
PANACEA PRODUCTS CORP : PANACEA PRODUCTS CORP-IMPORT
PREMIER PAINT ROLLER COMPANY L : PREMIER PAINT ROLLER/Z PRO
RAWLINGS SPORTING GOODS : RAWLINGS SPORT GOODS CO
RAY-O-VAC : RAYOVAC
RHINO SEED & LANDSCAPE SUPPLY : RHINO SEED & LANDSCAPING SUPPLY LLC
RITE IN THE RAIN : RITE IN THE RAIN/ J L DARLING LLC
SAFETY 1ST INC : SAFETY 1ST/DOREL """

[["DETEX", "Detex/Lockman Peck"], ['Falcon/Monarch', 'Falcon', 'Monarch'], ['Mckinney', 'McKinney Hinges'], ['Select', 'SELECT Hinges']]

[['DETEX', 'Detex/Lockman Peck'],
 ['Falcon/Monarch', 'Falcon', 'Monarch'],
 ['Mckinney', 'McKinney Hinges'],
 ['Select', 'SELECT Hinges']]